In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
import datetime
from pyspark.sql.functions import lit, unix_timestamp


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords1=nltk.corpus.stopwords.words('english')
stopwords2=nltk.corpus.stopwords.words('french')
stopwords3=nltk.corpus.stopwords.words('spanish')
stopwords=stopwords1+stopwords2+stopwords3

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
IP = 'localhost' 
PORT = 9003       
TIME = 6000

In [4]:
def regular_expression(word):
    if re.match('^#[a-zA-Z0-9]+',word) or re.match('^[a-zA-Z0-9]+',word):
        return True
    else:
        return False

In [5]:
def update_state(value,historyvalue):
    if historyvalue is None:
        historyvalue=0
    return sum(value,historyvalue)

In [6]:
def wordCount(words):
    counting=words.map(lambda w:w.lower()).filter(regular_expression).filter(lambda w:w not in stopwords).map(lambda w:(w,1)) \
    .updateStateByKey(update_state).transform(lambda wordmap:wordmap.sortBy(lambda word:word[1],ascending=False))
    return counting                                                                        

In [ ]:
if __name__ == '__main__':
    # Initialization
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")

    sql_context = SQLContext(sc)

    ssc = StreamingContext(sc, 5)
    ssc.checkpoint("~/checkpoint_TwitterApp")

    # listen
    dataStream = ssc.socketTextStream(IP, PORT)

    words = dataStream.flatMap(lambda line: line.split(" "))

    # word count
    wordCount = wordCount(words)
    wordCount.pprint(30)

    # start
    ssc.start()
    time.sleep(TIME)
    ssc.stop(stopSparkContext=False, stopGraceFully=True)
